# Biosemi Active II recorded file streaming

Resample and Save

In [5]:
# import mne
# import numpy as np
# from mne.channels import make_standard_montage
# import matplotlib.pyplot as plt
# from mne.datasets import eegbci

# raw = mne.io.read_raw_bdf("C:\\Users\\pipo_\\OneDrive\\Desktop\\test_data\\Testdata3.bdf")

# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage("biosemi64")

# raw.set_montage(montage, on_missing='ignore')

# raw_eeg = raw.copy().resample(512)

# raw_eeg.save("biosemi_SSVEP.fif", overwrite=True)

Load data for stream

In [6]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")
select_ch = ['O1','Oz','PO3','PO4','POz','Pz']
raw_eeg = raw_eeg.pick(select_ch)

Opening raw data file biosemi_SSVEP.fif...
    Range : 0 ... 100863 =      0.000 ...   196.998 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_30056\4293364053.py:8: RuntimeWarning: This filename (biosemi_SSVEP.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")


In [8]:
from mne_realtime import LSLClient, MockLSLStream
from pylsl import StreamInfo, StreamOutlet
import time
import mne

# Assuming raw_eeg is already created
# Create the LSL stream for the raw EEG data
eeg_info = StreamInfo(name='SomSom', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

def send_eeg_chunks(raw_eeg, chunk_size):
    """ Send raw EEG data in chunks through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array

    for start_idx in range(0, n_samples - chunk_size + 1):
        # Determine the end of the chunk with 1-sample overlapping
        end_idx = start_idx + chunk_size
        # Extract the chunk of data
        chunk = data[:, start_idx:end_idx]
        # Push the chunk of raw EEG data to the LSL stream
        eeg_outlet.push_chunk(chunk.T.tolist())  # Transpose for proper LSL format
        # print(f"Sent chunk of {end_idx - start_idx} samples starting at {start_idx} and ending at {end_idx}")
        time.sleep(1 / sfreq)

if __name__ == '__main__':
    while True:
        # Set the chunk size (e.g., 100 samples per chunk)
        chunk_size = 512
        # Send raw EEG data in chunks
        send_eeg_chunks(raw_eeg, chunk_size)

KeyboardInterrupt: 

In [1]:
# from mne_realtime import LSLClient, MockLSLStream
# from pylsl import StreamInfo, StreamOutlet
# import time
# import mne

# # Assuming raw_eeg is already created and events are detected
# # Find events in raw_eeg (already done in your case)
# events_raw = mne.find_events(raw_eeg, shortest_event=0)

# # Create the LSL stream for the trigger events (Markers)
# trigger_info = StreamInfo(name='TriggerStream', type='Markers', channel_count=1, nominal_srate=raw_eeg.info['sfreq'], channel_format='int32')
# trigger_outlet = StreamOutlet(trigger_info)

# # Host name and wait time
# host = 'My_mne_stream'
# wait_max = 5

# previous_event_time = 0

# def send_triggers(events_raw, sfreq):
#     global previous_event_time
#     """ Send trigger events through LSL based on event timestamps. """
#     for event in events_raw:
#         sample_idx = event[0]  # The sample index of the event
#         trigger_type = event[2]  # The event ID (trigger type)

#         # Convert the sample index to time in seconds
#         event_time = (sample_idx / sfreq)
        
#         # Simulate waiting until the event time
#         print("waiting to send event for: ",event_time-previous_event_time)
#         time.sleep(event_time-previous_event_time)
        
#         # Send the trigger over LSL
#         trigger_outlet.push_sample([trigger_type])
#         print(f"Sent trigger: {trigger_type} at time {event_time+previous_event_time:.3f} seconds")

#         previous_event_time = event_time

# if __name__ == '__main__':
#     # Start the EEG stream using MockLSLStream
#     with MockLSLStream(host, raw_eeg, 'eeg'):
#         # Start the client to receive the EEG stream
#         with LSLClient(info=raw_eeg.info, host=host, wait_max=wait_max) as client:
#             client_info = client.get_measurement_info()
            
#             # Send triggers in real-time based on the events found in the raw data
#             send_triggers(events_raw, raw_eeg.info['sfreq'])

#             # Infinite loop to simulate real-time streaming
#             while True:
#                 pass  # Keeps the stream alive
